In [30]:
from utils import load_mnist_data, Encoder
from multiviewdata.torchdatasets import SplitMNIST, NoisyMNIST
from torch.utils.data import Subset
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from cca_zoo.deepmodels import architectures
from cca_zoo.deepmodels import SplitAE
import pytorch_lightning as pl
# print("PyTorch Version: ",torch.__version__)
# print("Torchvision Version: ",torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  print("Using the GPU!")
else:
  print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")

Using the GPU!


In [26]:
train_loader, val_loader, train_labels, val_labels=load_mnist_data(n_train=50000)

In [31]:
for x in train_loader:
  print(x)
  break

{'views': [tensor([[[[-0.0277, -0.0128,  0.0529,  ..., -0.1071,  0.0297,  0.0685],
          [-0.0608,  0.2138, -0.0134,  ..., -0.0385,  0.2101, -0.0946],
          [-0.2048,  0.0058, -0.0760,  ...,  0.0263,  0.0108,  0.0236],
          ...,
          [ 0.0256, -0.0344,  0.0824,  ...,  0.1161, -0.0026,  0.0530],
          [ 0.1644, -0.1008, -0.0312,  ..., -0.0516, -0.0094,  0.0442],
          [-0.1476, -0.0927, -0.2258,  ..., -0.0992,  0.0943, -0.1539]]],


        [[[ 0.0534, -0.0844, -0.0668,  ..., -0.1067, -0.0145, -0.0138],
          [ 0.0480, -0.0720,  0.0509,  ..., -0.1103, -0.0909, -0.1467],
          [ 0.0272, -0.2718,  0.0696,  ..., -0.1862,  0.0602, -0.0708],
          ...,
          [-0.1228, -0.1551, -0.0610,  ..., -0.1972,  0.0978,  0.0144],
          [ 0.0133, -0.1500,  0.0536,  ...,  0.0904, -0.0208, -0.0930],
          [ 0.0982, -0.0591, -0.0140,  ..., -0.1193,  0.2122, -0.2344]]],


        [[[ 0.0908, -0.0025,  0.1479,  ...,  0.3053, -0.0750, -0.0797],
          [-0.0

In [27]:
encoder=architectures.CNNEncoder(latent_dims=32)
decoder_1=architectures.CNNDecoder(latent_dims=32)
decoder_2=architectures.CNNDecoder(latent_dims=32)

In [28]:
splitAE=SplitAE(latent_dims=32,encoder=encoder,decoders=[decoder_1,decoder_2])
trainer=pl.Trainer(max_epochs=20,accelerator='gpu', devices=1,log_every_n_steps=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(splitAE, train_loader)

C:\anaconda3\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:110: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  category=PossibleUserWarning,
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | encoder        | CNNEncoder | 25.1 K
1 | decoders       | ModuleList | 51.8 K
2 | latent_dropout | Dropout    | 0     
----------------------------------------------
77.0 K    Trainable params
0         Non-trainable params
77.0 K    Total params
0.308     Total estimated model params size (MB)
C:\anaconda3\envs\pytorch\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [38]:
def train_classifier(encoder, cls, dataloader, epochs=100, supervised=False):
    encoder=encoder.to(device)
    optimizer = optim.Adam(cls.parameters(), lr=0.001, weight_decay=1e-4)
    if supervised:
        optimizer = optim.Adam(list(cls.parameters())+list(encoder.parameters()), lr=0.001, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    loss_traj = []
    accuracy_traj = []

    for epoch in tqdm(range(epochs)):

        loss_epoch = 0
        corrects_epoch = 0
        for batch in dataloader:
            x,y=batch['views'][0],batch['label']

            batch_size = x.size(0)
            x = x.float()
            x,y=x.to(device),y.to(device)
            optimizer.zero_grad()
            feature = encoder(x.to(device))
            outs = cls(feature.view(batch_size,-1))
            loss = criterion(outs, y)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outs, 1)
            corrects_epoch += torch.sum(preds == y.data)
            loss_epoch += loss.detach()

        loss_traj.append(loss_epoch)
        epoch_acc = corrects_epoch.double() / len(dataloader.dataset)
        accuracy_traj.append(epoch_acc)

        if epoch % 10 == 0:
            print('Epoch {}, loss {:.3f}, train accuracy {}'.format(epoch, loss_epoch, epoch_acc))

    return cls, loss_traj

In [39]:
def test(encoder, cls, dataloader):
    encoder=encoder.to(device)
    cls.eval()

    corrects_epoch = 0
    for batch in dataloader:
        x,y=batch['views'][0],batch['label']

        x = x.float()
        batch_size = x.size(0)
        x, y = x.to(device), y.to(device)
        h = encoder(x).view(batch_size, -1)
        outs = cls(h)
        _, preds = torch.max(outs, 1)
        corrects_epoch += torch.sum(preds == y.data)

    epoch_acc = corrects_epoch.double() / len(dataloader.dataset)
    print('Test accuracy {}'.format(epoch_acc))

In [42]:
linear_cls = nn.Sequential(nn.Linear(32, 24),nn.ReLU(),nn.Linear(24,10)).to(device)
cls, loss_traj = train_classifier(splitAE.encoder, linear_cls, train_loader, epochs=40, supervised=False)
test(splitAE.encoder, cls,val_loader )

  2%|▎         | 1/40 [00:01<00:55,  1.42s/it]

Epoch 0, loss 787.783, train accuracy 0.7867000000000001


 28%|██▊       | 11/40 [00:14<00:38,  1.34s/it]

Epoch 10, loss 178.284, train accuracy 0.9463800000000001


 52%|█████▎    | 21/40 [00:27<00:25,  1.32s/it]

Epoch 20, loss 139.894, train accuracy 0.9583200000000001


 78%|███████▊  | 31/40 [00:39<00:10,  1.18s/it]

Epoch 30, loss 126.754, train accuracy 0.9612400000000001


100%|██████████| 40/40 [00:49<00:00,  1.25s/it]


Test accuracy 0.9617
